In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle

In [9]:
data=pd.read_csv('examples/human_integrators/output/integrators_log.csv')

In [10]:
data.head(10)

,Activity,Case ID,Hemoglobin,Sodium,Potassium,Dose,Responsive,Time
0,Exam,0,19.219235,143.987425,2.373778,NaN,True,1.169659
1,pot_int,0,NaN,NaN,NaN,1.126222,NaN,45.822124
2,Exam,0,19.219235,143.987425,3.500000,NaN,True,1.535216
3,hemo_nib,0,NaN,NaN,NaN,2.619235,NaN,47.173689
4,Exam,0,16.600000,143.987425,3.500000,NaN,True,0.221439
5,recovered,0,NaN,NaN,NaN,NaN,NaN,0.000000
6,Exam,1,16.221091,145.735689,4.684091,NaN,True,0.769231
7,sodium_inib,1,NaN,NaN,NaN,0.735689,NaN,47.674045
8,Exam,1,16.221091,145.000000,4.684091,NaN,True,0.764458
9,recovered,1,NaN,NaN,NaN,NaN,NaN,0.000000


In [15]:
data.drop('Responsive',axis=1)

,Activity,Case ID,Hemoglobin,Sodium,Potassium,Dose,Time
0,Exam,0,19.219235,143.987425,2.373778,NaN,1.169659
1,pot_int,0,NaN,NaN,NaN,1.126222,45.822124
2,Exam,0,19.219235,143.987425,3.500000,NaN,1.535216
3,hemo_nib,0,NaN,NaN,NaN,2.619235,47.173689
4,Exam,0,16.600000,143.987425,3.500000,NaN,0.221439
...,...,...,...,...,...,...,...
1517,pot_inib,99,NaN,NaN,NaN,0.014991,47.640727
1518,Exam,99,15.308706,144.064096,5.754840,NaN,1.492711
1519,pot_inib,99,NaN,NaN,NaN,0.012742,46.537836
1520,Exam,99,15.308706,144.064096,5.742098,NaN,1.645153


In [16]:
activities = ['Exam', 'hemo_int', 'hemo_inib',
       'sodium_int', 'sodium_inib', 'pot_int',
       'pot_inib', 'recovered',
       'Non responsive']

In [17]:
n_activities = len(activities)
map_activities = dict(zip(activities, range(n_activities)))


def zeros(n):
    return [0 for _ in range(n)]

def hot_encode_activity(activity):
    encode = zeros(n_activities+2)
    encode[map_activities[activity[0]]] = 1
    encode[-2] = activity[1]
    encode[-1] = activity[2]
    return encode
    
def activity_to_tensor(activity):
    return  torch.tensor([hot_encode_activity(activity)])

def hot_encode_trajectory(trajectory):
    return [[hot_encode_activity(activity)] for activity in trajectory]

def trajectory_to_tensor(trajectory):
    return torch.tensor(hot_encode_trajectory(trajectory), dtype= torch.float32 )

In [12]:
traces = [group.values.tolist() for _, group in data.groupby('Case ID')]

In [19]:
X = [traces.iloc[i][0] for i in range(len(traces))]
Y = [torch.tensor([int(x[-1][0]=="recovered")], dtype=torch.float) for x in X]
rX = [x if x[-1][0]!= "recovered" else x[:-1] for x in X]
trX = [trajectory_to_tensor(x) for x in rX]

AttributeError: 'list' object has no attribute 'iloc'